<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/PyTorch_Tutorial/0_QuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
from torch.utils.data import DataLoader #Dataset 순회
from torchvision import datasets #데이터
from torchvision.transforms import ToTensor

from torch import nn

In [14]:
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(train_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

In [25]:
#1. 신경망 구축
#부모 클래스를 상속받아 새로운 클래스를 만드는 것. nn.Module에는 파라미터 관리, GPU 이동 등 복잡한 기능이 담겨있다.
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__() #부모 클래스의 초기화 로직을 먼저 실행

    #신경망 정의
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )

  #신경망에 데이터를 어떻게 전달할지 순서 지정
  def forward(self, x):
    x = self.flatten(x)
    x = self.linear_relu_stack(x)
    return x

#2. 컴퓨터 환경 설정
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

#3. 모델 설정
model = NeuralNetwork().to(device) #데이터를 GPU로 보내는 PyTorch 메소드
print(model)

#4. 손실함수, 옵티마이저 설정
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [30]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device) #model과 같은 GPU에 있도록 to로 보내주기

    #예측, 손실계산
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward() #역전파: 오차를 사용해 가중치별 기울기 계산.
    optimizer.step() #최적화: optimizer공식 (SGD)에 따라 가중치 수정
    optimizer.zero_grad() #초기화: 이전 배치때 계산한 기울기가 다음 배치에 영향을 주지 않도록 지우는 작업.

    if batch%100 == 0:
      loss = loss.item()
      current = (batch+1)*len(X)
      print(f"loss : {loss}  [{current:>5d}/{size:>5d}]")

loss : 2.307480573654175  [   64/60000]
loss : 2.288498878479004  [ 6464/60000]
loss : 2.266786575317383  [12864/60000]
loss : 2.258558988571167  [19264/60000]
loss : 2.239301919937134  [25664/60000]
loss : 2.2175943851470947  [32064/60000]
loss : 2.2169950008392334  [38464/60000]
loss : 2.1807775497436523  [44864/60000]
loss : 2.1666438579559326  [51264/60000]
loss : 2.1478681564331055  [57664/60000]


In [31]:
def test(dataloader, model, loss_fn):
  #변수 초기화
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0


  model.eval() #model 평가모드 전환
  with torch.no_grad(): #가중치 업데이트 안할 것이니 기울기 계산 중지
    for X, y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)

      test_loss += loss_fn(pred, y).item() #item() : 텐서에서 값을 꺼내는 역할
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [34]:
epochs = 5
for t in range(epochs):
  print(f'Epoch {t+1}---------------------------------------------')
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print('끝')

Epoch 1---------------------------------------------
loss : 1.7133235931396484  [   64/60000]
loss : 1.6761806011199951  [ 6464/60000]
loss : 1.5346555709838867  [12864/60000]
loss : 1.595281720161438  [19264/60000]
loss : 1.4926300048828125  [25664/60000]
loss : 1.4786872863769531  [32064/60000]
loss : 1.481544017791748  [38464/60000]
loss : 1.4096648693084717  [44864/60000]
loss : 1.428530216217041  [51264/60000]
loss : 1.3312186002731323  [57664/60000]
Test Error: 
 Accuracy: 62.5%, Avg loss: 1.353679 

Epoch 2---------------------------------------------
loss : 1.4268561601638794  [   64/60000]
loss : 1.4043060541152954  [ 6464/60000]
loss : 1.2434622049331665  [12864/60000]
loss : 1.3363739252090454  [19264/60000]
loss : 1.2234026193618774  [25664/60000]
loss : 1.247617244720459  [32064/60000]
loss : 1.252118706703186  [38464/60000]
loss : 1.197562336921692  [44864/60000]
loss : 1.2233359813690186  [51264/60000]
loss : 1.136290192604065  [57664/60000]
Test Error: 
 Accuracy: 64.1%

In [36]:
torch.save(model.state_dict(), 'model.pth')

In [39]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [44]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
